In [ ]:
import os

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_file_path: Path

In [4]:
from cdclassifier.constants import *
from cdclassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            param_filepath = PARAMS_FILE_PATH
        ):
        
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self)->PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_file_path)
        create_directories([
            Path(config.tensorboard_root_log_dir),
            Path(model_ckpt_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_file_path= Path(config.checkpoint_model_file_path)
        )

        return prepare_callbacks_config


In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [21]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_file_path),
            save_best_only = True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    

In [22]:
from cdclassifier import logger

In [23]:

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    logger.exception(e)
    raise e 

[2024-08-18 19:33:17,584: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-18 19:33:17,587: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-18 19:33:17,587: INFO: common: created directory at: artifacts]
[2024-08-18 19:33:17,588: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2024-08-18 19:33:17,588: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
